# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846629091653                   -0.70    4.8         
  2   -7.852292290129       -2.25       -1.53    1.0   27.7ms
  3   -7.852611123377       -3.50       -2.55    1.8   31.8ms
  4   -7.852645951281       -4.46       -2.88    2.5   38.1ms
  5   -7.852646481349       -6.28       -3.14    1.0   27.8ms
  6   -7.852646678831       -6.70       -4.01    1.0   27.8ms
  7   -7.852646686585       -8.11       -5.13    1.8   33.9ms
  8   -7.852646686726       -9.85       -5.52    1.8   34.4ms
  9   -7.852646686723   +  -11.67       -5.39    1.5   33.5ms
 10   -7.852646686730      -11.20       -7.05    1.0   29.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846865747351                   -0.70           4.5         
  2   -7.852527656192       -2.25       -1.63   0.80    2.2    302ms
  3   -7.852637331248       -3.96       -2.71   0.80    1.0   27.7ms
  4   -7.852646453292       -5.04       -3.32   0.80    2.2   36.8ms
  5   -7.852646675102       -6.65       -4.16   0.80    1.5   31.0ms
  6   -7.852646686373       -7.95       -4.74   0.80    1.8   32.1ms
  7   -7.852646686720       -9.46       -5.54   0.80    1.8   33.2ms
  8   -7.852646686730      -11.04       -6.43   0.80    1.8   32.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.396035e+01     3.766948e+00
 * time: 0.4759640693664551
     1     1.316217e+00     1.779567e+00
 * time: 0.701862096786499
     2    -1.267434e+00     2.130194e+00
 * time: 0.7300810813903809
     3    -3.655668e+00     1.827853e+00
 * time: 0.7695000171661377
     4    -5.085818e+00     1.610329e+00
 * time: 0.809532880783081
     5    -6.756341e+00     8.795136e-01
 * time: 0.8488819599151611
     6    -7.433152e+00     2.866285e-01
 * time: 0.8885760307312012
     7    -7.634654e+00     1.774934e-01
 * time: 0.9161789417266846
     8    -7.737193e+00     1.473833e-01
 * time: 0.943321943283081
     9    -7.792022e+00     6.701587e-02
 * time: 0.9711670875549316
    10    -7.827638e+00     5.928948e-02
 * time: 0.9995729923248291
    11    -7.842692e+00     5.519228e-02
 * time: 1.0280780792236328
    12    -7.849212e+00     3.360990e-02
 * time: 1.0574209690093994
    13    -7.851440e+00     2.438694e-02
 * time: 1.0860199928283

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846815551532                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645904789                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.91s
  3   -7.852646686730      -13.30       -7.24    197ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.8117322311769275e-7
|ρ_newton - ρ_scfv| = 2.656208286506178e-7
|ρ_newton - ρ_dm|   = 2.194235965037396e-9
